# 📊 Sales Prediction Using Python

---

**Author:** Piyush Ramteke  
**Organization:** CodSoft  
**Task:** Sales Prediction Using Python (Task 2)  

---

## 🎯 Objective

Sales prediction involves forecasting the amount of a product that customers will purchase, taking into account various factors such as **advertising expenditure**, **target audience segmentation**, and **advertising platform selection**.

In this project, we will use **Interactive Visualizations** 📈 to:
1. **Explore** the Advertising dataset to understand relations between ad spending and sales
2. **Visualize** key patterns using **Plotly** for dynamic charts
3. **Build** robust regression models to predict sales
4. **Create an Interactive Widget** to simulate ad budget scenarios

## 📦 Dataset Overview

The dataset contains **200 records** of advertising budgets and sales:

| Feature | Description |
|---------|-------------|
| `TV` | Ad budget for TV (in thousands of $) |
| `Radio` | Ad budget for Radio (in thousands of $) |
| `Newspaper` | Ad budget for Newspaper (in thousands of $) |
| `Sales` | Product sales (in thousands of units) — **Target** |

---
## 1️⃣ Importing Libraries 📦

In [1]:
# Core Libraries
import numpy as np
import pandas as pd
import warnings

# Interactive Visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pyplot as plt

# Interactive Widgets
from ipywidgets import interact, widgets

# Modeling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

warnings.filterwarnings('ignore')
print('✅ All libraries loaded with Interactive Power! 🚀')

✅ All libraries loaded with Interactive Power! 🚀


---
## 2️⃣ Loading Data 📂

In [2]:
df = pd.read_csv('Advertising.csv')
print(f'📐 Shape: {df.shape}')
df.head()

📐 Shape: (200, 4)


,TV,Radio,Newspaper,Sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,12.0
3,151.5,41.3,58.5,16.5
4,180.8,10.8,58.4,17.9


---
## 3️⃣ Interactive EDA 🔍

In [3]:
# ── 3.1 Distribution of Features (Interactive) ────────────────

@interact(Column=['TV', 'Radio', 'Newspaper', 'Sales'])
def plot_distribution(Column):
    fig = px.histogram(df, x=Column, nbins=30, 
                       title=f'📊 Distribution of {Column}',
                       marginal='box', 
                       color_discrete_sequence=['#636EFA'])
    fig.show()

interactive(children=(Dropdown(description='Column', options=('TV', 'Radio', 'Newspaper', 'Sales'), value='TV'…

In [4]:
# ── 3.2 Sales vs Ad Spend (Interactive Trendlines) ────────────

fig = make_subplots(rows=1, cols=3, subplot_titles=('TV vs Sales', 'Radio vs Sales', 'Newspaper vs Sales'))

for i, col in enumerate(['TV', 'Radio', 'Newspaper'], 1):
    # Add scatter trace
    fig.add_trace(go.Scatter(x=df[col], y=df['Sales'], mode='markers', name=col), row=1, col=i)
    
fig.update_layout(title_text="📈 Sales vs Advertising Channels", showlegend=False)
fig.show()

In [5]:
# ── 3.3 Correlation Heatmap (Interactive) ─────────────────────

corr = df.corr()
fig = px.imshow(corr, text_auto=True, aspect="auto",
                title='🔥 Correlation Heatmap',
                color_continuous_scale='Viridis')
fig.show()

**💡 Insight:** **TV** has the strongest correlation with Sales, followed by **Radio**. **Newspaper** has a weaaak correlation.

In [6]:
# ── 3.4 3D Scatter Plot: TV, Radio & Sales ────────────────────

fig = px.scatter_3d(df, x='TV', y='Radio', z='Sales',
                    color='Sales', opacity=0.7,
                    title='🧊 3D View: Sales vs TV & Radio',
                    color_continuous_scale='Bluered')
fig.show()

---
## 4️⃣ Feature Engineering & Split ✂️

In [7]:
# Create Total Ad Spend
df['Total_Ad_Spend'] = df['TV'] + df['Radio'] + df['Newspaper']

# Select Features
X = df[['TV', 'Radio', 'Newspaper', 'Total_Ad_Spend']]
y = df['Sales']

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('✅ Data Split Successfully!')

✅ Data Split Successfully!


---
## 5️⃣ Model Training & Evaluation 🤖

In [8]:
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42)
}

results = {}
best_model = None
best_score = -np.inf

print('🚀 Training Models...\n')

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    results[name] = r2
    print(f'📌 {name:20} | R²: {r2:.4f} | RMSE: {rmse:.4f}')
    
    if r2 > best_score:
        best_score = r2
        best_model = model

print(f'\n🏆 Best Model: {best_model.__class__.__name__} with R²: {best_score:.4f}')

🚀 Training Models...

📌 Linear Regression    | R²: 0.9059 | RMSE: 1.7052
📌 Random Forest        | R²: 0.9066 | RMSE: 1.6986
📌 Gradient Boosting    | R²: 0.9275 | RMSE: 1.4964

🏆 Best Model: GradientBoostingRegressor with R²: 0.9275


In [9]:
# ── 5.1 Interactive Model Performance Comparison ──────────────

scores_df = pd.DataFrame(list(results.items()), columns=['Model', 'R2_Score'])

fig = px.bar(scores_df, x='R2_Score', y='Model', orientation='h',
             text='R2_Score', color='R2_Score',
             title='📊 Model R² Score Comparison',
             color_continuous_scale='RdYlGn')
fig.update_traces(texttemplate='%{text:.4f}', textposition='inside')
fig.show()

---
## 6️⃣ Interactive Prediction Simulator 🔮

Use the sliders below to adjust advertising budgets and predict sales in real-time!

In [10]:
def predict_sales(TV, Radio, Newspaper):
    total_spend = TV + Radio + Newspaper
    input_data = pd.DataFrame([[TV, Radio, Newspaper, total_spend]], 
                              columns=['TV', 'Radio', 'Newspaper', 'Total_Ad_Spend'])
    
    prediction = best_model.predict(input_data)[0]
    
    print(f'💰 Predicted Sales: {prediction:.2f} k units')
    print(f'📊 Total Investment: ${total_spend:.2f} k')

# Create Widget
interact(predict_sales, 
         TV=widgets.FloatSlider(min=0, max=300, step=5, value=150, description='📺 TV Ad:'),
         Radio=widgets.FloatSlider(min=0, max=100, step=2, value=40, description='📻 Radio Ad:'),
         Newspaper=widgets.FloatSlider(min=0, max=100, step=2, value=25, description='📰 News Ad:')
);

interactive(children=(FloatSlider(value=150.0, description='📺 TV Ad:', max=300.0, step=5.0), FloatSlider(value…

---